# Simulations of Survival rate

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import openrouteservice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Own Classes
from Simulation_Routing import *
from Simulating import *
from Simulation_Routing_Matrix import *
from Simulation_Routing_Matrix_copy import *
from Simulation_Routing_Matrix_Batch import *
from tqdm import tqdm

In [2]:
os.getcwd()

'C:\\Users\\Dillon\\Desktop\\AED-Route-Optimization-MDA-Project'

Initialize the class using the ip of the server.

In [3]:
ip = "35.159.16.25"

### New function version
Wrapped a function around so that we can better automate the simulations. Can also better keep track of what parameters we have specified and which have already been tried.

In [9]:
## NEW function version (old pre-function version below)

def simulation_run(ip=ip,              ## server IP, must request from Ruben (or others)
                   all=False,           ##
                   num_sim = 3,        ## number of simulation runs to perform for the given parameter values
                   max_num_resp = 100, ## maximum number of responders that can be considered
                   open_hr = 14.0,     ## given opening hour (choose one of 2.0, 8.0, 14.0, 20.0)
                   prop_cfr = 0.01,    ## proportion of the total population who are CFRs, determines the number sampled
                   aed_filter = ["Yes"]): ## which AEDs to consider (choose None to allow all)
    Simulations = Simulation(ip, all)
    RoutingMatrixCopy = RoutingSimulationMatrixBatch(ip)

    ## OTHER PARAMETERS determined by our function input
    
    # Likelihood of a responder declining the call to action
    # varies according to whether it is the day, evening, or night
    if 7.0 <= open_hr < 17.0:
        decline_rate = 0.7
    elif 17.0 <= open_hr < 23.0:
        decline_rate = 0.65
    else:
        decline_rate = 0.75
    
    # time of the day
    time_of_day = "day" if 8.0 <= open_hr <= 18.0 else "night"
    
    df_final = pd.DataFrame()

    print("Input values: \n opening_hour: " + str(open_hr) + 
      "\n decline_rate: " + str(decline_rate) + 
      "\n max_number_responder: " + str(max_num_resp) + 
      "\n filter_values: " + str(aed_filter) +
      "\n time_of_day: " + str(time_of_day) +
      "\n proportion: " + str(prop_cfr))

    x = 1
    df_final = []
    while x <= num_sim:
        try:
            df = Simulations.simulation_run(
                decline_rate, max_num_resp, open_hr,
                aed_filter, time_of_day, prop_cfr
            )
            df['filter_values'] = ','.join(aed_filter) if isinstance(aed_filter, list) else aed_filter
            df['opening_hour'] = open_hr
            df['decline_rate'] = decline_rate
            df['max_number_responder'] = max_num_resp
            df['proportion_of_CFR'] = prop_cfr
            # Rename all columns except 'patient_loc'
            suffix = f"_run{x}"
            df_renamed = df.rename(
                columns={col: f"{col}{suffix}" for col in df.columns if col != 'patient_loc'}
            )
    
            # Merge into the master DataFrame
            df_final.append(df_renamed)
    
        except Exception as e:
            print(f"[Warning] Simulation {x} failed: {e}. Retrying...")

        x+=1

    ## combine the run results with previous runs' results
    df_final = pd.concat(df_final, ignore_index=True)
    
    ## Build a string-safe version of parameters
    aed_str = "all_aeds" if aed_filter is None else "-".join(aed_filter)
    filename = (
        f"sim_results_"
        f"openhr{open_hr}_"
        f"cfr{prop_cfr}_"
        f"maxresp{max_num_resp}_"
        f"numsim{num_sim}_"
        f"aed{aed_str}.csv"
    )
    
    # Save the results
    df_final.to_csv("Data/" + filename, index=False)
    print(f"Results saved to {filename}")

    

In [10]:
opening_hour_list = [2.0, 8.0, 14.0, 20.0]

for i in opening_hour_list:
    simulation_run(prop_cfr = 0.01, all = True, open_hr=i, aed_filter=None)



Input values: 
 opening_hour: 2.0
 decline_rate: 0.75
 max_number_responder: 100
 filter_values: None
 time_of_day: night
 proportion: 0.01


100%|██████████| 388/388 [04:18<00:00,  1.50it/s]


Results saved to sim_results_openhr2.0_cfr0.01_maxresp100_numsim3_aedall_aeds.csv
Input values: 
 opening_hour: 8.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: None
 time_of_day: day
 proportion: 0.01


100%|██████████| 388/388 [04:38<00:00,  1.39it/s]


Results saved to sim_results_openhr8.0_cfr0.01_maxresp100_numsim3_aedall_aeds.csv
Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: None
 time_of_day: day
 proportion: 0.01


100%|██████████| 388/388 [06:03<00:00,  1.07it/s]


Results saved to sim_results_openhr14.0_cfr0.01_maxresp100_numsim3_aedall_aeds.csv
Input values: 
 opening_hour: 20.0
 decline_rate: 0.65
 max_number_responder: 100
 filter_values: None
 time_of_day: night
 proportion: 0.01


100%|██████████| 388/388 [05:27<00:00,  1.19it/s]

Results saved to sim_results_openhr20.0_cfr0.01_maxresp100_numsim3_aedall_aeds.csv


In [48]:
sim_low_cfr = pd.read_csv("Data/sim_results_openhr14.0_cfr0.005_maxresp100_numsim3_aedYes.csv")
mean_mean_prob_resp = (sim_low_cfr["prob_resp_run1"].mean() + sim_low_cfr["prob_resp_run2"].mean() + sim_low_cfr["prob_resp_run3"].mean())/3
mean_mean_prob_vec = (sim_low_cfr["prob_vec_run1"].mean() + sim_low_cfr["prob_vec_run2"].mean() + sim_low_cfr["prob_vec_run3"].mean())/3
print(mean_mean_prob_resp, mean_mean_prob_vec)

sim_low_cfr.loc[sim_low_cfr["aed_loc_run1"]!= "No AED", "aed_loc_run1"].value_counts().sum() + sim_low_cfr.loc[sim_low_cfr["aed_loc_run2"]!= "No AED", "aed_loc_run2"].value_counts().sum() + sim_low_cfr.loc[sim_low_cfr["aed_loc_run3"]!= "No AED", "aed_loc_run3"].value_counts().sum()

0.3513180503057621 0.18279808744468248


141

In [49]:
sim_old = pd.read_csv("Data/sim_results_openhr14.0_cfr0.03_maxresp100_numsim3_aedYes.csv")
(sim_old["prob_resp_run1"].mean() + sim_old["prob_resp_run2"].mean() + sim_old["prob_resp_run3"].mean())/3
mean_mean_prob_resp = (sim_old["prob_resp_run1"].mean() + sim_old["prob_resp_run2"].mean() + sim_old["prob_resp_run3"].mean())/3
print(mean_mean_prob_resp, mean_mean_prob_vec)

sim_old.loc[sim_old["aed_loc_run1"]!= "No AED", "aed_loc_run1"].value_counts().sum() + sim_old.loc[sim_old["aed_loc_run2"]!= "No AED", "aed_loc_run2"].value_counts().sum() + sim_old.loc[sim_old["aed_loc_run3"]!= "No AED", "aed_loc_run3"].value_counts().sum()

0.40128715971742457 0.18279808744468248


130

In [65]:
sim_tues = pd.read_csv("Data/sim_results_openhr14.0_cfr0.015_maxresp100_numsim3_aedYes.csv")
mean_mean_prob_resp = (sim_tues["prob_resp_run1"].mean() + sim_tues["prob_resp_run2"].mean() + sim_tues["prob_resp_run3"].mean())/3
print(mean_mean_prob_resp)

sim_tues.loc[sim_tues["aed_loc_run1"]!= "No AED", "aed_loc_run1"].value_counts().sum() + sim_tues.loc[sim_tues["aed_loc_run2"]!= "No AED", "aed_loc_run2"].value_counts().sum() 
+ sim_tues.loc[sim_tues["aed_loc_run3"]!= "No AED", "aed_loc_run3"].value_counts()




0.391328388211901


aed_loc_run3
(4.7046611, 50.8810353)    15
(4.7346682, 50.8825443)    13
(4.676586, 50.8718562)      4
(4.692543, 50.86957)        4
(4.7433965, 50.8674498)     4
(4.7116128, 50.867677)      3
(4.7063333, 50.9258595)     2
(4.678102, 50.858007)       2
(4.7247521, 50.8495093)     1
Name: count, dtype: int64

In [66]:
sim_tues

,patient_loc,responder_loc_run1,duration_Responder_run1,Indirect_Responder_loc_run1,aed_loc_run1,duration_AED_run1,vector_loc_run1,duration_Vector_run1,prob_vec_run1,prob_resp_run1,...,duration_AED_run3,vector_loc_run3,duration_Vector_run3,prob_vec_run3,prob_resp_run3,filter_values_run3,opening_hour_run3,decline_rate_run3,max_number_responder_run3,proportion_of_CFR_run3
0,"(4.724486, 50.875134)","(4.7247596578637125, 50.87510096634083)",7.65,No AED,No AED,10000.0,"(4.7243436, 50.8696549)",641.37,0.324247,0.715235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(4.663458, 50.872624)","(4.661517702308046, 50.87186836337241)",136.38,No AED,No AED,10000.0,"(4.6658621, 50.8774603)",621.19,0.335943,0.615328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(4.706287, 50.868107)",No responder,10000.00,No AED,No AED,10000.0,"(4.6998331, 50.8723536)",618.41,0.337587,0.337587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(4.729334, 50.886379)",No responder,10000.00,No AED,No AED,10000.0,"(4.7243436, 50.8696549)",1978.53,0.030983,0.030983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(4.7093, 50.870361)",No responder,10000.00,No AED,No AED,10000.0,"(4.6998331, 50.8723536)",750.80,0.267560,0.267560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,"(4.692054, 50.883744)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10000.00,"(4.6953981, 50.8785355)",612.09,0.341354,0.681126,Yes,14.0,0.7,100.0,0.015
1160,"(4.676329, 50.867263)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10000.00,"(4.6658621, 50.8774603)",1466.98,0.076075,0.417932,Yes,14.0,0.7,100.0,0.015
1161,"(4.676351, 50.870025)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,375.32,"(4.6658621, 50.8774603)",934.56,0.193768,0.744201,Yes,14.0,0.7,100.0,0.015
1162,"(4.71405, 50.873746)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10000.00,"(4.7243436, 50.8696549)",693.89,0.295680,0.295680,Yes,14.0,0.7,100.0,0.015


In [71]:
sim_tues

,patient_loc,responder_loc_run1_x,duration_Responder_run1_x,Indirect_Responder_loc_run1_x,aed_loc_run1_x,duration_AED_run1_x,vector_loc_run1_x,duration_Vector_run1_x,prob_vec_run1_x,prob_resp_run1_x,...,duration_AED_run3_y,vector_loc_run3_y,duration_Vector_run3_y,prob_vec_run3_y,prob_resp_run3_y,filter_values_run3_y,opening_hour_run3_y,decline_rate_run3_y,max_number_responder_run3_y,proportion_of_CFR_run3_y
0,"(4.724486, 50.875134)","(4.7247596578637125, 50.87510096634083)",7.65,No AED,No AED,10000.00,"(4.7243436, 50.8696549)",641.37,0.324247,0.715235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(4.663458, 50.872624)","(4.661517702308046, 50.87186836337241)",136.38,No AED,No AED,10000.00,"(4.6658621, 50.8774603)",621.19,0.335943,0.615328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(4.706287, 50.868107)",No responder,10000.00,No AED,No AED,10000.00,"(4.6998331, 50.8723536)",618.41,0.337587,0.337587,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(4.729334, 50.886379)",No responder,10000.00,No AED,No AED,10000.00,"(4.7243436, 50.8696549)",1978.53,0.030983,0.030983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(4.7093, 50.870361)",No responder,10000.00,No AED,No AED,10000.00,"(4.6998331, 50.8723536)",750.80,0.267560,0.267560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,"(4.715394, 50.88124)","(4.716029670431035, 50.881245534728656)",21.40,No AED,No AED,10000.00,"(4.6953981, 50.8785355)",1264.25,0.108605,0.512469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,"(4.738901, 50.882895)","(4.737592980138326, 50.88303288410458)",83.11,"(4.732240034768137, 50.88091632340358)","(4.7346682, 50.8825443)",620.50,"(4.7243436, 50.8696549)",1548.46,0.065933,0.657869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,"(4.710779, 50.883194)","(4.710431632242269, 50.88334756209112)",23.49,"(4.704577722930277, 50.8809925272623)","(4.7046611, 50.8810353)",480.77,"(4.6953981, 50.8785355)",1023.74,0.165680,0.760797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,"(4.690987, 50.92475)","(4.690596468436933, 50.92567491713022)",54.54,No AED,No AED,10000.00,"(4.6658621, 50.8774603)",4996.65,0.000155,0.073929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
sim_tues_1 = sim_tues.loc[0:337,:]
sim_tues_2 = sim_tues.loc[338:675,:]
sim_tues_3 = sim_tues.loc[676:,:]

sim_tues = sim_tues_1.merge(sim_tues_2, on = "patient_loc", how = "left")


In [74]:
sim_tues = sim_tues.drop(columns = ['responder_loc_run2_x', 'duration_Responder_run2_x',
       'Indirect_Responder_loc_run2_x', 'aed_loc_run2_x',
       'duration_AED_run2_x', 'vector_loc_run2_x', 'duration_Vector_run2_x',
       'prob_vec_run2_x', 'prob_resp_run2_x', 'filter_values_run2_x',
       'opening_hour_run2_x', 'decline_rate_run2_x',
       'max_number_responder_run2_x', 'proportion_of_CFR_run2_x',
       'responder_loc_run3_x', 'duration_Responder_run3_x',
       'Indirect_Responder_loc_run3_x', 'aed_loc_run3_x',
       'duration_AED_run3_x', 'vector_loc_run3_x', 'duration_Vector_run3_x',
       'prob_vec_run3_x', 'prob_resp_run3_x', 'filter_values_run3_x',
       'opening_hour_run3_x', 'decline_rate_run3_x',
       'max_number_responder_run3_x', 'proportion_of_CFR_run3_x',
       'responder_loc_run1_y', 'duration_Responder_run1_y',
       'Indirect_Responder_loc_run1_y', 'aed_loc_run1_y',
       'duration_AED_run1_y', 'vector_loc_run1_y', 'duration_Vector_run1_y',
       'prob_vec_run1_y', 'prob_resp_run1_y', 'filter_values_run1_y',
       'opening_hour_run1_y', 'decline_rate_run1_y',
       'max_number_responder_run1_y', 'proportion_of_CFR_run1_y', 'responder_loc_run3_y', 'duration_Responder_run3_y',
       'Indirect_Responder_loc_run3_y', 'aed_loc_run3_y',
       'duration_AED_run3_y', 'vector_loc_run3_y', 'duration_Vector_run3_y',
       'prob_vec_run3_y', 'prob_resp_run3_y', 'filter_values_run3_y',
       'opening_hour_run3_y', 'decline_rate_run3_y',
       'max_number_responder_run3_y', 'proportion_of_CFR_run3_y'])

In [75]:
sim_tues.columns

Index(['patient_loc', 'responder_loc_run1_x', 'duration_Responder_run1_x',
       'Indirect_Responder_loc_run1_x', 'aed_loc_run1_x',
       'duration_AED_run1_x', 'vector_loc_run1_x', 'duration_Vector_run1_x',
       'prob_vec_run1_x', 'prob_resp_run1_x', 'filter_values_run1_x',
       'opening_hour_run1_x', 'decline_rate_run1_x',
       'max_number_responder_run1_x', 'proportion_of_CFR_run1_x',
       'responder_loc_run2_y', 'duration_Responder_run2_y',
       'Indirect_Responder_loc_run2_y', 'aed_loc_run2_y',
       'duration_AED_run2_y', 'vector_loc_run2_y', 'duration_Vector_run2_y',
       'prob_vec_run2_y', 'prob_resp_run2_y', 'filter_values_run2_y',
       'opening_hour_run2_y', 'decline_rate_run2_y',
       'max_number_responder_run2_y', 'proportion_of_CFR_run2_y'],
      dtype='object')

In [76]:
sim_tues

,patient_loc,responder_loc_run1_x,duration_Responder_run1_x,Indirect_Responder_loc_run1_x,aed_loc_run1_x,duration_AED_run1_x,vector_loc_run1_x,duration_Vector_run1_x,prob_vec_run1_x,prob_resp_run1_x,...,duration_AED_run2_y,vector_loc_run2_y,duration_Vector_run2_y,prob_vec_run2_y,prob_resp_run2_y,filter_values_run2_y,opening_hour_run2_y,decline_rate_run2_y,max_number_responder_run2_y,proportion_of_CFR_run2_y
0,"(4.724486, 50.875134)","(4.7247596578637125, 50.87510096634083)",7.65,No AED,No AED,10000.00,"(4.7243436, 50.8696549)",641.37,0.324247,0.715235,...,10000.0,"(4.7243436, 50.8696549)",641.37,0.324247,0.703309,Yes,14.0,0.7,100.0,0.015
1,"(4.663458, 50.872624)","(4.661517702308046, 50.87186836337241)",136.38,No AED,No AED,10000.00,"(4.6658621, 50.8774603)",621.19,0.335943,0.615328,...,10000.0,"(4.6658621, 50.8774603)",621.19,0.335943,0.631270,Yes,14.0,0.7,100.0,0.015
2,"(4.706287, 50.868107)",No responder,10000.00,No AED,No AED,10000.00,"(4.6998331, 50.8723536)",618.41,0.337587,0.337587,...,10000.0,"(4.6998331, 50.8723536)",618.41,0.337587,0.337587,Yes,14.0,0.7,100.0,0.015
3,"(4.729334, 50.886379)",No responder,10000.00,No AED,No AED,10000.00,"(4.7243436, 50.8696549)",1978.53,0.030983,0.030983,...,10000.0,"(4.7243436, 50.8696549)",1978.53,0.030983,0.030983,Yes,14.0,0.7,100.0,0.015
4,"(4.7093, 50.870361)",No responder,10000.00,No AED,No AED,10000.00,"(4.6998331, 50.8723536)",750.80,0.267560,0.267560,...,10000.0,"(4.6998331, 50.8723536)",750.80,0.267560,0.267560,Yes,14.0,0.7,100.0,0.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,"(4.715394, 50.88124)","(4.716029670431035, 50.881245534728656)",21.40,No AED,No AED,10000.00,"(4.6953981, 50.8785355)",1264.25,0.108605,0.512469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,"(4.738901, 50.882895)","(4.737592980138326, 50.88303288410458)",83.11,"(4.732240034768137, 50.88091632340358)","(4.7346682, 50.8825443)",620.50,"(4.7243436, 50.8696549)",1548.46,0.065933,0.657869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,"(4.710779, 50.883194)","(4.710431632242269, 50.88334756209112)",23.49,"(4.704577722930277, 50.8809925272623)","(4.7046611, 50.8810353)",480.77,"(4.6953981, 50.8785355)",1023.74,0.165680,0.760797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,"(4.690987, 50.92475)","(4.690596468436933, 50.92567491713022)",54.54,No AED,No AED,10000.00,"(4.6658621, 50.8774603)",4996.65,0.000155,0.073929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

# ## OLD pre-function

# ip = "35.159.16.25"
# Simulations = Simulation(ip)
# RoutingMatrixCopy = RoutingSimulationMatrixBatch(ip)

# # Number of times the simulations are run to later get an average survivability rate
# NO_SIMULATIONS = 6

# ## PARAMETERS WE SET
# # Maximal number of responder contacted
# max_number_responder = 100
# # Time of the incident
# ## choose one of the following: 2.0, 8.0, 14.0, 20.0
# opening_hour = 14.0
# # proportion of CFR of the total (ex.: 0.01 = 1%)
# proportion = 0.01
# # list of values that are in the available column
# filter_values = ["Yes"]

# ## OTHER PARAMETERS
# # Likelihood of a responder declining the call to action
# # varies according to whether it is the day, evening, or night
# if 7.0 <= opening_hour < 17.0:
#     decline_rate = 0.7
# elif 17.0 <= opening_hour < 23.0:
#     decline_rate = 0.65
# else:
#     decline_rate = 0.75

# # time of the day
# time_of_day = "day" if 8.0 <= opening_hour <= 18.0 else "night"

# df_final = pd.DataFrame()

# print("Input values: \n opening_hour: " + str(opening_hour) + 
#       "\n decline_rate: " + str(decline_rate) + 
#       "\n max_number_responder: " + str(max_number_responder) + 
#       "\n filter_values: " + str(filter_values) +
#       "\n time_of_day: " + str(time_of_day) +
#       "\n proportion: " + str(proportion))

# df_final = pd.DataFrame()

# x = 0
# df_final = []
# while x < NO_SIMULATIONS:
#     try:
#         df = Simulations.simulation_run(
#             decline_rate, max_number_responder, opening_hour,
#             filter_values, time_of_day, proportion
#         )
#         df['filter_values'] = ','.join(filter_values) if isinstance(filter_values, list) else filter_values
#         df['opening_hour'] = opening_hour
#         df['decline_rate'] = decline_rate
#         df['max_number_responder'] = max_number_responder
#         df['proportion_of_CFR'] = proportion
#         # Rename all columns except 'patient_loc'
#         suffix = f"_run{x+1}"
#         df_renamed = df.rename(
#             columns={col: f"{col}{suffix}" for col in df.columns if col != 'patient_loc'}
#         )

#         # Merge into the master DataFrame
#         df_final.append(df_renamed)

#     except Exception as e:
#         print(f"[Warning] Simulation {x+1} failed: {e}. Retrying...")

# df_final = pd.concat(df_final, ignore_index=True)


Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 100
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.01


  2%|▏         | 9/388 [00:09<06:30,  1.03s/it]


KeyboardInterrupt: 

In [30]:
# SAVE THE DATAFRAME!!!
df.to_csv("Data/sim_runs/YOUR_FILENAME_HERE.csv")

## Debugging


In [29]:
## PARAMETERS WE SET
# Maximal number of responder contacted
max_number_responder = 1000
# Time of the incident
## choose one of the following: 2.0, 8.0, 14.0, 20.0
opening_hour = 14.0
# proportion of CFR of the total (ex.: 0.01 = 1%)
proportion = 0.02


print("Input values: \n opening_hour: " + str(opening_hour) + 
      "\n decline_rate: " + str(decline_rate) + 
      "\n max_number_responder: " + str(max_number_responder) + 
      "\n filter_values: " + str(filter_values) +
      "\n time_of_day: " + str(time_of_day) +
      "\n proportion: " + str(proportion))

df = Simulations.simulation_run(
            decline_rate, max_number_responder, opening_hour,
            filter_values, time_of_day, proportion
        )


Input values: 
 opening_hour: 14.0
 decline_rate: 0.7
 max_number_responder: 1000
 filter_values: ['Yes']
 time_of_day: day
 proportion: 0.02


100%|██████████| 388/388 [10:08<00:00,  1.57s/it]


In [30]:
# SAVE THE DATAFRAME!!!
df.to_csv("Data/sim_runs/debug_run_3.csv")

In [32]:
df_1 = pd.read_csv("Data/sim_runs/debug_run.csv")
df_2 = pd.read_csv("Data/sim_runs/debug_run_2.csv")

In [39]:
df["responder_loc"].value_counts()

responder_loc
No responder                               112
(4.726162579635614, 50.87427682579571)       4
(4.728076759091802, 50.8862301174485)        3
(4.6901339607145145, 50.89294406168314)      2
(4.729748648752874, 50.900059324561035)      2
                                          ... 
(4.662973834193056, 50.86121395811966)       1
(4.702060059438565, 50.88841222083602)       1
(4.724808214661289, 50.92769673861054)       1
(4.708022109575111, 50.8769896670177)        1
(4.730412749638898, 50.93259863929417)       1
Name: count, Length: 239, dtype: int64